In [4]:
"""
generate train data, validation data, and test data
"""
import os
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.ndimage import imread
from scipy.misc import imsave
from scipy.misc import imresize
import numpy as np
import os
from numpy import genfromtxt
import tensorflow as tf
from PIL import Image
import pickle
import helper
from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import collections
import time

# Create Train and Target Data

In [70]:
# get codes
gallery_features_batch1 = np.load(open(r'gait_data/relu6_1253/gallery_codes_batch_1', mode='rb'))
probe_features_batch1 = np.load(open(r'gait_data/relu6_1253/probe_codes_batch_1', mode='rb'))
# feature shape
print("gallery length: {}".format(gallery_features_batch1.shape))
print("gallery shape: {}".format(gallery_features_batch1[0].shape))
print("")
print("probe length: {}".format(probe_features_batch1.shape))
print("probe shape: {}".format(probe_features_batch1[0].shape))

gallery length: (1253,)
gallery shape: (32, 4096)

probe length: (1253,)
probe shape: (31, 4096)


### Add Zero

In [71]:
gallery_features_batch1_add0 = np.zeros([len(gallery_features_batch1), 43, 4096], dtype=float)
probe_features_batch1_add0 = np.zeros([len(probe_features_batch1), 43, 4096], dtype=float)

g_seq_length = []
p_seq_length = []
for ii, (gallery, probe) in enumerate(zip(gallery_features_batch1, probe_features_batch1)):
    gallery_length = gallery.shape[0]
    probe_length = probe.shape[0]
    g_seq_length.append(gallery_length)
    p_seq_length.append(probe_length)
    
    gallery_features_batch1_add0[ii, :gallery_length, :] = gallery
    probe_features_batch1_add0[ii, :probe_length, :] = probe

### Create Target 1 Data

In [5]:
batch_1_target_1 = np.concatenate((gallery_features_batch1_add0, probe_features_batch1_add0), axis=1)
target_1 = np.ones([batch_1_target_1.shape[0]])

In [6]:
print("batch_1_target_1:", batch_1_target_1.shape)
print("")
print("target_1:", target_1.shape)

batch_1_target_1: (1253, 86, 4096)

target_1: (1253,)


### Create Target 0 Data
We create a same length batch with shuffled data
直到检测shuffle的idx没有重叠，函数会返回ok，否则打印error

In [10]:
idx = np.arange(len(gallery_features_batch1_add0))
np.random.shuffle(idx)
idx

array([ 941,  715,  239, ..., 1197, 1046,  410])

In [11]:
def exam_shuffle_idx(idx):
    x =None
    for ii, index in enumerate(idx):
        if ii == index:
            x ='error'
            print(x)
            break
    if x != 'error':
        print("ok")
    return None
exam_shuffle_idx(idx)

ok


In [12]:
# create shuffled half_probe_features_batch1
shuffled_probe_features_batch1 = np.zeros(probe_features_batch1_add0.shape)
print(shuffled_probe_features_batch1.shape)
for ii, index in enumerate(idx):
    shuffled_probe_features_batch1[ii, :, :] = probe_features_batch1_add0[index]

(1253, 43, 4096)


In [13]:
batch_1_target_0 = np.concatenate((gallery_features_batch1_add0, shuffled_probe_features_batch1), axis=1)
target_0 = np.zeros([batch_1_target_0.shape[0]])

In [14]:
batch_1_target_0[0][43:]

array([[ 0.        ,  0.        ,  0.00745186, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03079182,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.00188977, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
batch_1_target_1[941][43:]

array([[ 0.        ,  0.        ,  0.00745186, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03079182,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.00188977, ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
# Concatenate half_batch_1_target_0 and half_batch_1_target_1 together
new_batch_1 = np.concatenate((batch_1_target_1, batch_1_target_0), axis=0)
new_batch_1.shape

(2506, 86, 4096)

In [17]:
with open('gait_data/relu6_1253/ordered_new_batch_1', 'wb') as f:
    np.save(f, new_batch_1)

In [19]:
target_1_and_0 = np.concatenate((target_1, target_0))
new_batch_1[2385]

array([[ 0.        ,  0.01815505,  0.02952662, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.01808102,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.01243681,  0.        , ...,  0.        ,
         0.00709434,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [21]:
# Shuffle new_half_batch_1 to random order
idx2 = np.arange(len(new_batch_1))
np.random.shuffle(idx2)
idx2

array([1492, 1092, 1208, ..., 1214, 1494,  465])

In [23]:
shuffled_new_batch_1 = np.ndarray(new_batch_1.shape, dtype=float)
shuffled_target_1_and_0 = np.ndarray(target_1_and_0.shape, dtype=float)
for ii, index in enumerate(idx2):
    shuffled_new_batch_1[ii, :, :] = new_batch_1[index]
    shuffled_target_1_and_0[ii] = target_1_and_0[index]

In [26]:
with open('gait_data/relu6_1253/shuffled_new_batch_1', 'wb') as f:
    np.save(f, shuffled_new_batch_1)

In [29]:
one_hot_shuffled_new_batch_1_target = np.ndarray([2506, 2], dtype=float)
for ii, each in enumerate(shuffled_target_1_and_0):
    if int(each) == 1:
        one_hot_shuffled_new_batch_1_target[ii, :] = 1, 0
    elif int(each) == 0:
        one_hot_shuffled_new_batch_1_target[ii, :] = 0, 1

In [37]:
with open('gait_data/relu6_1253/one_hot_shuffled_new_batch_1_target', 'wb') as f:
    np.save(f, one_hot_shuffled_new_batch_1_target)

## Create seq_length

In [102]:
p_seq_length = np.array(p_seq_length)
shuffled_p_seq_length = np.zeros(p_seq_length.shape, dtype=int)
for ii, index in enumerate(idx):
    shuffled_p_seq_length[ii] = p_seq_length[index]

In [103]:
shuffled_p_seq_length.shape

(1253,)

In [104]:
seq_column1 = np.reshape(np.concatenate([g_seq_length, g_seq_length]), [2506, 1])
seq_column2 = np.reshape(np.concatenate([g_seq_length, shuffled_p_seq_length]), [2506, 1])
seq_length = np.concatenate([seq_column1, seq_column2], axis=1)

shuffled_seq_length = np.zeros(seq_length.shape, dtype=int)
for ii, index in enumerate(idx2):
    shuffled_seq_length[ii] = seq_length[index]

In [107]:
with open('gait_data/relu6_1253/shuffled_seq_length', 'wb') as f:
    np.save(f, shuffled_seq_length)

# Checkpoints 1

In [5]:
seq_length = np.load(open(r'gait_data/relu6_1253/shuffled_seq_length', mode='rb'))
shuffled_new_batch_1 = np.load(open(r'gait_data/relu6_1253/shuffled_new_batch_1', mode='rb'))
one_hot_shuffled_new_batch_1_target = np.load(open(r'gait_data/relu6_1253/one_hot_shuffled_new_batch_1_target', mode='rb'))
print(shuffled_new_batch_1.shape)

(2506, 86, 4096)


In [6]:
one_hot_shuffled_new_batch_1_target

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 1.,  0.]])

# Get Train and Validation data

In [7]:
print("data length: {}".format(len(shuffled_new_batch_1)))
print("every length has : {}".format(shuffled_new_batch_1.shape))

data length: 2506
every length has : (2506, 86, 4096)


In [8]:
split_frac = 0.8
split_num = int(split_frac*len(shuffled_new_batch_1))

train_x_raw, val_x_raw = shuffled_new_batch_1[:split_num], shuffled_new_batch_1[split_num:]
train_y_raw, val_y_raw = one_hot_shuffled_new_batch_1_target[:split_num], one_hot_shuffled_new_batch_1_target[split_num:]
train_seq_length, val_seq_length = seq_length[:split_num], seq_length[split_num:]

# print size
print("Train_x: {}".format(np.array(train_x_raw).shape))
print("Validation_x: {}".format(np.array(val_x_raw).shape))
print("Train_y: {}".format(np.array(train_y_raw).shape))
print("Validation_y: {}".format(np.array(val_y_raw).shape))
print("train_seq_length: {}".format(np.array(train_seq_length).shape))
print("val_seq_length: {}".format(np.array(val_seq_length).shape))

Train_x: (2004, 86, 4096)
Validation_x: (502, 86, 4096)
Train_y: (2004, 2)
Validation_y: (502, 2)
train_seq_length: (2004, 2)
val_seq_length: (502, 2)


# Build the graph

In [9]:
def build_rnn(batch_size=20, lstm_size=1000, num_layers=1, learning_rate=0.001, grad_clip=1):
    
    # reset graph
    tf.reset_default_graph()
    
    with tf.name_scope('inputs_1'):
        inputs_1 = tf.placeholder(tf.float32, [None, None, 4096], name="inputs_1")
    with tf.name_scope('inputs_2'):
        inputs_2 = tf.placeholder(tf.float32, [None, None, 4096], name="inputs_2")
    with tf.name_scope('targets'):
        target_ = tf.placeholder(tf.float32, [None, 2], name="targets")
        
    seq_1 = tf.placeholder(tf.float32, (None,), name="seq_1")
    
    seq_2 = tf.placeholder(tf.float32, (None,), name="seq_2")
    
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
#     seq_length = tf.placeholder(tf.int32, (None, ), name='seq_length')
    with tf.variable_scope("LSTM"):
#         with tf.name_scope("RNN_cells_1"):
            # Your basic LSTM cell
        #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
        #         lstm = tf.nn.relu(lstm)
        lstm_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_1 = tf.contrib.rnn.DropoutWrapper(lstm_1, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_1 = tf.contrib.rnn.MultiRNNCell([drop_1] * num_layers)

#         with tf.name_scope("RNN_init_state_1"):
            # Getting an initial state of all zeros
        initial_state_1 = cell_1.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_1"):
        outputs_1, final_state_1 = tf.nn.dynamic_rnn(cell_1, inputs_1, initial_state=initial_state_1, 
                                                     sequence_length=seq_1)

    with tf.variable_scope("LSTM", reuse=True):
#         with tf.name_scope("RNN_cells_2"):
            # Your basic LSTM cell
    #         lstm = tf.contrib.rnn.LSTMBlockFusedCell(lstm_size, forget_bias=1.0, use_peephole=True)
    #         lstm = tf.nn.relu(lstm)
        lstm_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size)
            # Add dropout to the cell
        drop_2 = tf.contrib.rnn.DropoutWrapper(lstm_2, output_keep_prob=keep_prob)
            # Stack up multiple LSTM layers, for deep learning
        cell_2 = tf.contrib.rnn.MultiRNNCell([drop_2] * num_layers)

#         with tf.name_scope("RNN_init_state_2"):
            # Getting an initial state of all zeros
        initial_state_2 = cell_2.zero_state(batch_size, tf.float32)

#         with tf.name_scope("RNN_forward_2"):
        outputs_2, final_state_2 = tf.nn.dynamic_rnn(cell_2, inputs_2, initial_state=initial_state_2, 
                                                     sequence_length=seq_2)
        
    with tf.name_scope('predictions'):
        # only choose final one
#         final_output_concat = tf.concat([outputs_1[:, -1], outputs_2[:, -1]], 1)
        final_output_concat = outputs_1[:, -1] - outputs_2[:, -1]
        logits = tf.contrib.layers.fully_connected(final_output_concat, 2)
        predictions = tf.nn.softmax(logits, name='predictions')
        tf.summary.histogram('predictions', predictions)
        
    with tf.name_scope('cost'):
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target_, logits=logits))
        tf.summary.scalar('cost', cost)
        
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip, name='clip')
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
#         optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
    with tf.name_scope('saver'):
        saver = tf.train.Saver()
        
    merged = tf.summary.merge_all()
    
    export_nodes = ['inputs_1', 'inputs_2', 'target_', 'initial_state_1', 'initial_state_2', 'outputs_1', 'outputs_2',
                    'final_state_1', 'final_state_2', 'keep_prob', 'cost', 'logits', 'predictions', 'optimizer', 
                    'merged', 'saver', 'learning_rate', 'cell_1', 'cell_2', 'final_output_concat', 'seq_1', 'seq_2']
    
    Graph = collections.namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Get Batches

In [10]:
def get_batches(data, target, seq_length, batch_size):
    n_batches = len(data)//batch_size
    data = data[:batch_size * n_batches]
    target = target[:batch_size * n_batches]
    for ii in range(0, batch_size*n_batches, batch_size):
        data_batch = data[ii:ii + batch_size]
        target_batch = target[ii:ii + batch_size]
        seq_length_batch_1 = seq_length[ii:ii + batch_size, 0]
        seq_length_batch_2 = seq_length[ii:ii + batch_size, 1]
        
        yield data_batch, target_batch, seq_length_batch_1, seq_length_batch_2

# Accuracy

# Train

In [11]:
train_x = train_x_raw
val_x = val_x_raw
train_y = train_y_raw
val_y = val_y_raw

In [15]:
def train(model, epochs, file_writer, save_string):

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        iteration = 1
        mean_val_Acc = 0
        mean_val_loss = 0
        count_Acc_not_increase_epochs = 0
        count_loss_not_decrease_epochs = 0
        Last_val_Acc = 0
        Last_val_loss = 0
        
        for e in range(epochs):
            for train_batch, target_batch, seq_length_batch_1, seq_length_batch_2 \
                             in get_batches(train_x, train_y, train_seq_length, batch_size):
                    
                start = time.time()
                
#                 state_1 = sess.run(model.initial_state_1)
#                 state_2 = sess.run(model.initial_state_2)
#                 print(train_batch[:, :15, :].shape)
#                 print(target_batch)
    #             print('target: {}'.format(target_batch.shape))
                feed = {model.inputs_1: train_batch[:, :43, :],
                        model.inputs_2: train_batch[:, 43:, :],
                        model.target_: target_batch,
                        model.keep_prob: 1,
#                         model.initial_state_1: state_1,
#                         model.initial_state_2: state_2,
                        model.learning_rate: lr,
                        model.seq_1: seq_length_batch_1,
                        model.seq_2: seq_length_batch_2
                        }
#                 print(model.outputs_1.shape, model.outputs_1[:, -1], model.final_output_concat, 
#                       model.logits, model.predictions, model.cost)
                loss, _, logi, pred, summary = sess.run([model.cost, model.optimizer, model.logits, model.predictions, model.merged], \
                                              feed_dict=feed)
                print(model.outputs_1, model.outputs_2)
                print(pred)
                print(logi)
#                 print('pred: {}'.format(type(pred)))
    #             print('pred: {}'.format(np.count_nonzero(pred)))
    #             print('pred: {}'.format(np.sum(pred>0.1)))
    #             print('pred: {}'.format(pred.size))
    #             print('output: {}'.format(out[1, 39, :]))
                
                if iteration%5==0:
                    end = time.time()
#                     acc = calculate_accuracy(sess, pred, target_batch, sq_length)
                    print("Epoch: {}/{},".format(e+1, epochs),' ',
                          "Iteration: {},".format(iteration),' ',
                          "Train loss: {:.3f},".format(loss),' ',
                          "{:.1f}s /batch.".format((end-start)/5),' '
#                           "ACCuracy: %{:.3f}".format(acc)
                         )
                    
                    file_writer.add_summary(summary, iteration)
                
#                 if iteration%25==0:
#                     validation_loss = []
#                     validation_Acc = []
                    
#                     if batch_size >= len(val_x):
#                         val_batch_size = 30
#                     else: 
#                         val_batch_size = batch_size
                    
#                     for val_batch, val_target_batch in get_batches(val_x, val_y, val_batch_size):

#                         val_state_1 = sess.run(model.cell_1.zero_state(val_batch_size, tf.float32))
#                         val_state_2 = sess.run(model.cell_2.zero_state(val_batch_size, tf.float32))

#                         feed = {model.inputs_1: val_batch[:, :15, :],
#                                 model.inputs_2: val_batch[:, 15:, :],
#                                 model.target_: val_target_batch,
#                                 model.keep_prob: 1,
#                                 model.initial_state_1: val_state_1,
#                                 model.initial_state_2: val_state_2,
#                                }

#                         val_loss, val_pred = sess.run([model.cost, model.predictions], feed_dict=feed)
                        
# #                         val_acc = calculate_accuracy(sess, val_pred, val_target_batch, val_sq_length)

#                         validation_loss.append(val_loss)
                        
# #                         validation_Acc.append(val_acc)
                        
#                     Last_val_Acc = mean_val_Acc
#                     Last_val_loss = mean_val_loss
                        
#                     mean_val_loss = sum(validation_loss)/len(validation_loss)  
#                     mean_val_Acc = sum(validation_Acc)/len(validation_Acc)
#                     print()
#                     print("Validation loss: {:.3f},".format(mean_val_loss), ' ',
#                           "Validation Accuracy: %{:.3f}".format(mean_val_Acc))
#                     print()
                    
                iteration += 1
                
            # Early stopping  
#             if mean_val_Acc - Last_val_Acc <= -0.3:
#                 count_Acc_not_increase_epochs += 1
#             if Last_val_loss - mean_val_loss <= -0.01:
#                 count_loss_not_decrease_epochs += 1
            
#             if count_Acc_not_increase_epochs >= 5:
#                 break
#             if count_loss_not_decrease_epochs >= 5:
#                 break
                
        model.saver.save(sess, r"{}".format(save_string))

In [16]:
!mkdir checkpoints_identification

mkdir: checkpoints_identification: File exists


In [17]:
epochs = 1000
batch_size = 100
for lstm_size in [2000]:
    for num_layers in [2]:
        for lr in [0.001]:
                log_string = 'logs/4/lr={},nl={},ls={},bs={}'.format(lr, num_layers, lstm_size, batch_size)
                save_string = 'checkpoints\lr={}_nl={}_ls={}_bs={}.ckpt'.format(lr, num_layers, lstm_size, batch_size)
                
                writer = tf.summary.FileWriter(log_string)

                model = build_rnn(batch_size=batch_size, lstm_size=lstm_size, 
                                  num_layers=num_layers, learning_rate=lr, grad_clip=2)

                train(model, epochs, writer, save_string)
                
                print(' ')
                print(' ')
                print("leraning_rate={},num_layers={},lstm_size={},batch_size={} finished, saved".format(lr, num_layers, lstm_size, batch_size))
                print(' ')
                print(' ')

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0.5  0.5]
 [ 0

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.95897073  0.04102928]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471]
 [ 0.50014526  0.49985471

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 ]
 [ 0.50034148  0.4996585 

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  ]
 [ 0.50028706  0.499713  

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539]
 [ 0.50028461  0.49971539

...

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609]
 [ 0.50084388  0.49915609

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287]
 [ 0.50086713  0.49913287

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.98000836  0.01999158]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574]
 [ 0.5009042   0.49909574

Tensor("LSTM/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32) Tensor("LSTM_1/rnn/transpose:0", shape=(100, ?, 2000), dtype=float32)
[[ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223]
 [ 0.5010078   0.49899223

KeyboardInterrupt: 